In [1]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Research essay A0284714Y.pdf")

documents = loader.load()


/Users/crazygynkz/Desktop/KAYO/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

textSplitting = RecursiveCharacterTextSplitter(
    chunk_size = 600, chunk_overlap = 0, 
    length_function = len, is_separator_regex = False
)

chunks = textSplitting.split_documents(documents)

print(len(chunks))

33


In [3]:
from sentence_transformers import SentenceTransformer
import numpy


embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

chunk_texts = [chunk.page_content for chunk in chunks]

embeddings = embedding_model.encode(chunk_texts, show_progress_bar=True)

print(embeddings)

embeddings_np = numpy.array(embeddings).astype('float32') #fass takes in 



/Users/crazygynkz/Desktop/KAYO/.venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/crazygynkz/Desktop/KAYO/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 2/2 [00:04<00:00,  2.10s/it]

[[ 0.00761158  0.03599565 -0.03254024 ...  0.07229744 -0.03705115
  -0.00895973]
 [ 0.01605107 -0.04350835 -0.02358476 ...  0.00184933 -0.05110533
  -0.01226669]
 [ 0.02385614  0.03040851 -0.0315797  ... -0.02918818 -0.03505195
   0.03309226]
 ...
 [-0.04611548 -0.02328918  0.01801944 ... -0.01524004 -0.09549437
   0.10284238]
 [ 0.00610571 -0.00023818 -0.01129274 ...  0.00765599 -0.09609738
   0.07402455]
 [ 0.00658685  0.01320795 -0.02266516 ... -0.0151975  -0.03260812
   0.03156128]]


In [4]:
import faiss #facebook ai similarity search 

query = "tourism in india"

query_embedding = embedding_model.encode(query, show_progress_bar=True)

query_embedding_np = numpy.array([query_embedding]).astype('float32')

nearest_neighbours = 5

dimension = embeddings_np.shape[1]

index = faiss.IndexFlatL2(dimension)
 
index.add(embeddings_np)

distance, indices = index.search(query_embedding_np, nearest_neighbours)

#The lower the distance, the more similar the chunk is to your query.
#hese are the positions of the most similar chunks within your original chunks list

print("Distance: ", distance)


print("Indices: ", indices)

similarity_scores = [ 1 / (1 + dist) for dist in distance[0] ]

print("Similarity scores: ", similarity_scores)


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.40it/s]

Distance:  [[0.620383   0.64027786 0.65519524 0.70657516 0.7070992 ]]
Indices:  [[25 21  3 15  5]]
Similarity scores:  [0.6171380377698183, 0.6096528050717577, 0.6041583362045819, 0.5859689196334956, 0.5857890393323103]


In [5]:
#Retrieving the chunks using distance and indices

for i in range(len(indices)) :
    for rank in range(nearest_neighbours):
        chunk_index = indices[i][rank]
        similarity = similarity_scores[rank]
        chunk_text = chunks[chunk_index]
        print(f"Rank {rank+1}: {chunk_text.page_content} with similarity {similarity}")
        

Rank 1: Tourism has played a pivotal role in strengthening the multifaceted relationship between India and Singapore. The thriving tourism industry has fostered political, economic, and cultural ties between the two nations. Through increased people-to-people exchanges, tourism has built trust, understanding, and goodwill, positively influencing diplomatic relations. Economically, the tourism sector generates substantial revenue, employment, and business opportunities, reinforcing the interdependence between the two countries. Culturally, tourism has served as a medium for showcasing and preserving with similarity 0.6171380377698183
Rank 2: helping to preserve and celebrate invaluable cultural heritage, further strengthening the ties between Singapore and India.  Addressing Challenges, Opportunities, Future Prospects and Recommendations:   While promoting tourism between Singapore and India holds immense potential, it is crucial to address the challenges and capitalise on opportunities

In [6]:
#LLM 
import os
import google.generativeai as genai

api_key = os.getenv("GENAI_API_KEY")
if not api_key:
    raise ValueError("API key not found in environment variables.")

genai.configure(api_key=api_key)

llm = genai.GenerativeModel('gemini-1.5-pro')

response = llm.generate_content("What is life")

print(response.text)

That's a question that has philosophers, scientists, and theologians pondering for centuries! There's no single, universally accepted answer, but here are some perspectives:

**Scientifically:**

* **Biology:** Life is defined as a self-sustaining system capable of growth, reproduction, adaptation, metabolism (converting energy), and response to stimuli.  This definition focuses on the processes and characteristics that distinguish living things from non-living matter.
* **Chemistry:** Life arises from complex interactions of organic molecules like proteins, nucleic acids, carbohydrates, and lipids. Understanding the chemical basis of life is crucial for fields like medicine and genetics.

**Philosophically:**

* **Meaning & Purpose:** Many philosophical inquiries revolve around the meaning and purpose of life, often exploring questions of consciousness, morality, and existence. 
* **Experience & Subjectivity:**  Some philosophers emphasize the subjective experience of being alive, foc

In [19]:
prompt = " Please answer the following question in context of the provided text:\n"
prompt += "Question: " + query + "\n"
for i in range(len(indices)) :
    for rank in range(nearest_neighbours):
        chunk_index = indices[i][rank]
        similarity = similarity_scores[rank]
        chunk_text = chunks[chunk_index]
        prompt += "Context: " + chunk_text.page_content + "\n"
        
response = llm.generate_content(prompt)

print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The provided text focuses primarily on the role of tourism in deepening the bilateral relationship between Singapore and India. While it doesn't directly delve into the specifics of tourism in India, it does mention several key aspects related to India's tourism sector:\n\n* **India's \"Incredible India\" campaign (2002):** This campaign was launched to raise global awareness of India's diverse tourism offerings. It contributed to the growth of India's tourism industry.\n* **India's inbound tourism growth:** The text highlights the remarkable growth in foreign tourist arrivals (FTAs) to India in 2023, with a 64% increase compared to the previous year.\n* **India as a key tourism destination for Singaporeans:**  The text mentions that India, with its rich herita